In [ ]:
import datetime
import io
import os

import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from PIL import Image
from tqdm import tqdm

from satellite.utils import get_landsat_collection

In [ ]:
project_name = os.getenv("EE_PROJECT_NAME")
assert project_name, "Please set the EE_PROJECT_NAME environment variable"
print(f"Initializing Earth Engine with project {project_name}")
ee.Initialize(project=project_name)

In [ ]:
# Define the region of interest (Los Angeles bounding box, source: chatGPT)
roi = ee.Geometry.Rectangle([-118.67, 33.7, -118.15, 34.35])
# define the year range
start_year = 2000
end_year = 2023

for year in tqdm(range(start_year, end_year + 1)):
    # Get Landsat data for the year
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)
    landsat_data = get_landsat_collection(roi, start_date, end_date)
    # Compute NDVI
    ndvi = landsat_data.normalizedDifference(["NIR", "RED"]).rename(f"NDVI_{year}")
    # Generate the NDVI image
    output_dir = "earthengine_outputs/"
    output_fpath = os.path.join(output_dir, f"ndvi_{year}.tif")
    os.makedirs(output_dir, exist_ok=True)
    geemap.ee_export_image(
        ndvi, filename=output_fpath, scale=30, region=roi, file_per_band=False
    )

    # Display the NDVI image on the map
    if year in {start_year, end_year}:
        print(f"Displaying NDVI for year {year} for sanity check")
        Map = geemap.Map()
        Map.centerObject(roi, 10)
        Map.addLayer(
            ndvi,
            {"min": -1, "max": 1, "palette": ["blue", "white", "green"]},
            f"NDVI_{year}",
        )
        display(Map)

In [ ]:
# Directory containing the exported TIFF files
output_gif = "ndvi_time_series.gif"
# Get a sorted list of TIFF files
tiff_fpaths = sorted(
    [
        os.path.join(output_dir, fname)
        for fname in os.listdir(output_dir)
        if fname.endswith(".tif")
    ]
)

# Create a list to store images for the GIF
gif_frames = []

# Define NDVI visualization range
vmin, vmax = -1, 1

# Loop through each TIFF and generate frames
for tiff_fpath in tiff_fpaths:

    # Open the TIFF file
    with rasterio.open(tiff_fpath) as src:
        # Read the NDVI data
        ndvi = src.read(1)

        # Plot the NDVI using Matplotlib
        plt.figure(figsize=(8, 6))
        plt.imshow(ndvi, cmap="RdYlGn", vmin=vmin, vmax=vmax)
        plt.colorbar(label="NDVI")
        year = tiff_fpath.split("_")[-1].split(".")[0]
        plt.title(f"NDVI - {year}")
        plt.axis("off")

        # Save the plot to a buffer
        buf = io.BytesIO()
        plt.savefig(buf, format="png", bbox_inches="tight")
        plt.close()
        buf.seek(0)

        # Load the PNG from buffer into Pillow and add to GIF frames
        gif_frames.append(Image.open(buf))

# Save frames as a GIF
gif_frames[0].save(
    output_gif,
    save_all=True,
    append_images=gif_frames[1:],
    duration=1000,  # Duration per frame in milliseconds
    loop=0,  # Infinite loop
)

print(f"GIF saved as {output_gif}")

In [ ]:
# List to store average NDVI values
avg_ndvi_values = []

# Loop through each TIFF and calculate the average NDVI
for tiff_fpath in tiff_fpaths:
    # Open the TIFF file
    with rasterio.open(tiff_fpath) as src:
        # Read the NDVI data
        ndvi = src.read(1)
        # Calculate the average NDVI, ignoring NaN values
        avg_ndvi = np.nanmean(ndvi)
        avg_ndvi_values.append(avg_ndvi)

# Extract years from the file names
years = [int(tiff_fpath.split("_")[-1].split(".")[0]) for tiff_fpath in tiff_fpaths]

# Plot the average NDVI over time
plt.figure(figsize=(10, 6))
plt.plot(years, avg_ndvi_values, marker="o", linestyle="-", color="b")
plt.xlabel("Year")
plt.ylabel("Average NDVI")
plt.title("Average NDVI Over Time")
plt.grid(True)
plt.savefig("average_ndvi_over_time.png")
plt.show()
